## Processamento dos dados

In [70]:
import pandas as pd
import numpy as np

In [71]:
df_movies = pd.read_table('movies.csv', sep=',')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [72]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [73]:
df_ratings = pd.read_table('ratings.csv', sep=',', keep_default_na=False)
df_ratings['userId'] = df_ratings['userId'].apply(str)
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [74]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  object 
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 3.1+ MB


In [75]:
df = df_movies.merge(df_ratings, how='left', on='movieId')

## Perguntas a serem respondidas

1. Quantos gêneros o filme "The Matrix" tem? Quais são os gêneros?
2. Quantos gêneros cada filme tem, em média?
3. Quais são os cinco filmes com maior número de gêneros?
4. Quantos usuários deram nota para o filme "The Matrix"?
5. Quantos usuários, em média, deram nota para cada filme?
6. Quais são os cinco filmes para os quais mais usuários deram nota?
7. Qual o rating médio do filme "The Matrix"?
8. Cada filme tem um rating médio. Qual a média desses ratings, por gênero?
9. Para cada gênero de filme, quantos usuários deram cada nota possível?

Dicas:
1. Só é necessário as tabelas movies e ratings para este projeto.
2. Pesquise a função "explode" no pandas. Ela pode ser útil.

### 1. Quantos gêneros o filme "The Matrix" tem? Quais são os gêneros?

In [76]:
df_matrix = df[df['title'].str.contains('Matrix')]
df_matrix.head()

,movieId,title,genres,userId,rating,timestamp
45015,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1,5.0,9.649819e+08
45016,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,4,1.0,9.451734e+08
45017,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,10,0.5,1.455356e+09
45018,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,13,5.0,9.874570e+08
45019,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,15,4.0,1.510572e+09


#### O ID do filme The Matrix é 2571

In [77]:
df_matrix = df[df['movieId'] == 2571]

qty_genres_matrix = len(df_matrix['genres'].values[0].split('|'))
print('Quantidade de generos no filme Matrix:', qty_genres_matrix)

genres_matrix = df_matrix['genres'].values[0].split('|')
print('Quais os generos do file Matrix:', genres_matrix)

Quantidade de generos no filme Matrix: 3
Quais os generos do file Matrix: ['Action', 'Sci-Fi', 'Thriller']


### 2. Quantos gêneros cada filme tem, em média?

In [78]:
# Criando uma coluna chamada "qty_genres" a partir da coluna "genres":
df['qty_genres'] = df['genres'].str.split("|").str.len()

In [79]:
genres_by_movie = df.groupby(['title']).mean(['qty_genres'])

In [80]:
print('O DataFrame a seguir mostra a quantidade de generos por filme:')
pd.DataFrame(genres_by_movie.qty_genres).reset_index()

O DataFrame a seguir mostra a quantidade de generos por filme:


,title,qty_genres
0,'71 (2014),4.0
1,'Hellboy': The Seeds of Creation (2004),5.0
2,'Round Midnight (1986),2.0
3,'Salem's Lot (2004),4.0
4,'Til There Was You (1997),2.0
...,...,...
9732,eXistenZ (1999),3.0
9733,xXx (2002),3.0
9734,xXx: State of the Union (2005),3.0
9735,¡Three Amigos! (1986),2.0


### 3. Quais são os cinco filmes com maior número de gêneros?

In [81]:
most_qty_genre = df.sort_values(by='qty_genres', ascending=False)
most_qty_genre = most_qty_genre[:5]
most_qty_genre = most_qty_genre[['title', 'qty_genres']]

In [82]:
print('Os cinco filmes com maior número de gêneros são:')
most_qty_genre.head()

Os cinco filmes com maior número de gêneros são:


,title,qty_genres
91910,Rubber (2010),10
91911,Rubber (2010),10
78231,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,8
92178,Tangled (2010),7
36314,Mulan (1998),7


In [83]:
print('Obs: Possuem outros filmes com 7 generos, além desses três acima.')

Obs: Possuem outros filmes com 7 generos, além desses três acima.


### 4. Quantos usuários deram nota para o filme "The Matrix"?

In [84]:
# Criando um dataframe apenas com os filmes que contêm o nome "Matrix":
matrix_movie = df.loc[df['title'].str.contains("Matrix", case = True)]

In [85]:
# Adaptando o dataframe e deixando apenas o filme "Matrix, The (1999)"
matrix_movie = matrix_movie[matrix_movie['title'] == 'Matrix, The (1999)']

In [86]:
matrix_movie

,movieId,title,genres,userId,rating,timestamp,qty_genres
45015,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1,5.0,9.649819e+08,3
45016,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,4,1.0,9.451734e+08,3
45017,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,10,0.5,1.455356e+09,3
45018,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,13,5.0,9.874570e+08,3
45019,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,15,4.0,1.510572e+09,3
...,...,...,...,...,...,...,...
45288,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,603,5.0,9.631776e+08,3
45289,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,606,5.0,1.171326e+09,3
45290,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,607,5.0,9.630800e+08,3
45291,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,608,5.0,1.117162e+09,3


In [87]:
print('O filme "The Matrix" recebeu nota de {} usuários.'.format(matrix_movie['userId'].unique().shape[0]))

O filme "The Matrix" recebeu nota de 278 usuários.


### 5. Quantos usuários, em média, deram nota para cada filme?

In [88]:
df_movie_avg_rating = df.drop_duplicates(['movieId', 'userId'])
df_movie_avg_rating = df_movie_avg_rating.groupby('movieId').count()
df_movie_avg_rating = df_movie_avg_rating[['userId']]
avg_rating = df_movie_avg_rating['userId'].sum() / df_movie_avg_rating['userId'].count()
avg_rating = round(avg_rating, 2)

In [89]:
print(f'Em média, {avg_rating} usuários deram nota para cada filme.')

Em média, 10.35 usuários deram nota para cada filme.


### 6. Quais são os cinco filmes para os quais mais usuários deram nota?

In [90]:
# Criando um dataframe agrupando por títulos dos filmes e contando as entradas da coluna "userId":
filmes_qty_ratings = pd.DataFrame(
    df.groupby('title')['userId'].count()).sort_values(by=['userId'], ascending=False).head().reset_index()

filmes_qty_ratings['index'] = [1, 2, 3, 4, 5]
filmes_qty_ratings.set_index('index', inplace = True)

In [91]:
print('Os cinco filmes para os quais mais usuários deram nota foram:')
filmes_qty_ratings

Os cinco filmes para os quais mais usuários deram nota foram:


,title,userId
index,,
1,Forrest Gump (1994),329
2,"Shawshank Redemption, The (1994)",317
3,Pulp Fiction (1994),307
4,"Silence of the Lambs, The (1991)",279
5,"Matrix, The (1999)",278


### 7. Qual o rating médio do filme "The Matrix"?

In [92]:
df_matrix_ratings_avg = df[df['movieId'] == 2571]
df_matrix_ratings_avg = df_matrix_ratings_avg[['rating']]
df_matrix_ratings_avg = df_matrix_ratings_avg.values.mean()
df_matrix_ratings_avg = round(df_matrix_ratings_avg, 2)

In [93]:
print('O rating médio do filme "The Matrix" é de', df_matrix_ratings_avg)

O rating médio do filme "The Matrix" é de 4.19


### 8. Cada filme tem um rating médio. Qual a média desses ratings, por gênero?

In [94]:
# Criando uma cópia do dataframe original e incluindo nele a coluna "generos" que vai separar cada gênero da coluna "genres":
df_sep_by_genres = df
df_sep_by_genres['sep_genres'] = df_sep_by_genres.genres.str.split('|')
df_sep_by_genres = df_sep_by_genres.explode('sep_genres')

In [95]:
df_sep_by_genres.head(3)

,movieId,title,genres,userId,rating,timestamp,qty_genres,sep_genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703.0,5,Adventure
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703.0,5,Animation
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703.0,5,Children


In [96]:
print('O DataFrame a seguir mostra a média de ratings que cada gênero recebeu:')
pd.DataFrame(df_sep_by_genres.groupby('sep_genres')['rating'].mean().round(2))

O DataFrame a seguir mostra a média de ratings que cada gênero recebeu:


,rating
sep_genres,
(no genres listed),3.49
Action,3.45
Adventure,3.51
Animation,3.63
Children,3.41
Comedy,3.38
Crime,3.66
Documentary,3.80
Drama,3.66


### 9. Para cada gênero de filme, quantos usuários deram cada nota possível?

In [97]:
df_genre = df
df_genre['genres_split'] = df_genre['genres'].str.split('|')
df_genre = df_genre.explode('genres_split')
df_genero_each = df_genre.groupby(['genres_split', 'rating'])['userId'].count()

In [98]:
print('O DataFrame a seguir mostra quantos usuários deram cada nota possível para cada gênero:')
pd.DataFrame(df_genero_each).head(50)

O DataFrame a seguir mostra quantos usuários deram cada nota possível para cada gênero:


userId
genres_split       rating        
(no genres listed) 0.5          2
                   1.0          2
                   2.0          2
                   2.5          6
                   3.0          6
                   3.5          6
                   4.0          8
                   4.5          8
                   5.0          7
Action             0.5        449
                   1.0        904
                   1.5        577
                   2.0       2548
                   2.5       1777
                   3.0       6331
                   3.5       4153
                   4.0       7678
                   4.5       2468
                   5.0       3750
Adventure          0.5        306
                   1.0        627
                   1.5        415
                   2.0       1769
                   2.5       1352
                   3.0       4838
                   3.5       3285
                   4.0       6392
                   4.5       2027
                   5.0       3150
Animation          0.5         80
                   1.0        116
                   1.5         96
                   2.0        346
                   2.5        365
                   3.0       1279
                   3.5       1051
                   4.0       1988
                   4.5        682
                   5.0        985
Children           0.5        169
                   1.0        301
                   1.5        161
                   2.0        721
                   2.5        530
                   3.0       2054
                   3.5       1205
                   4.0       2358
                   4.5        648
                   5.0       1061
Comedy             0.5        632